In [1]:
import pandas as pd

In [2]:
from data_extraction.dummy_data_extractor import extract_dummy_data

In [4]:
master_table = extract_dummy_data("dummy_data")

C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


In [6]:
print(master_table.head())

                                             pointID unit dqType  \
0  @p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...   °C  Nulls   
1  @p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...   °C  Nulls   

                     dqStart      dqDuration   pointInterval  \
0  2023-03-12 01:05:00+04:00 1 days 11:10:00 0 days 00:05:00   
1  2023-03-19 01:10:00+04:00 0 days 23:30:00 0 days 00:05:00   

                          features  \
0  [p:dmc_All:r:2de337c0-72b69972]   
1  [p:dmc_All:r:2de337c0-72b69972]   

                                                 his  
0                                  DMC Building 1...  
1                                  DMC Building 1...  


In [8]:
his_data = master_table['his']

In [10]:
print(his_data.head())

0                                    DMC Building 1...
1                                    DMC Building 1...
Name: his, dtype: object


In [2]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from data_extraction.dummy_data_extractor import extract_dummy_data

def prophet_forecasting(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Function to perform time series forecasting using Prophet.

    Parameters:
    - df: DataFrame, input data with columns ['ts', 'temp', 'new_point']
    - length_of_missing_data: int, length of missing data
    - data_logging_interval: int, data logging interval
    - dqStart: not used in the function but kept for consistency with the original signature

    Returns:
    - df: DataFrame with predictions for all rows with missing columns. Index names as ts.
    """
    df = extract_dummy_data("dummy_data")
    df = df.at[0, "his"]
    mt = df.set_index(["ts"])

    # Tag and filter rows with missing
    mt["status"] = mt.isna().any(axis=1)
    mt_predict = mt[mt["status"] == 1]
    X_predict = mt_predict.iloc[:, 0:1]

    # Filtered master table
    mt_train = mt.dropna()

    # Separate data for temperature
    df_temp = mt_train[['temp']].reset_index().rename(columns={'ts': 'ds', 'temp': 'y'})

    # Ensure 'ds' column is timezone-naive
    df_temp['ds'] = df_temp['ds'].dt.tz_localize(None)

    # Calculate periods based on length_of_missing_data and data_logging_interval
    periods = int(length_of_missing_data / data_logging_interval) + 1

    # Split the data into training and testing sets
    train_temp, test_temp = train_test_split(df_temp, test_size=0.2, shuffle=False)

    # Initialize Prophet model with tuned hyperparameters
    model_temp = Prophet(seasonality_mode='additive', interval_width=0.95, changepoint_prior_scale=0.00001)

    # Fit the model
    model_temp.fit(train_temp)

    # Create future DataFrame for temp
    future_temp = model_temp.make_future_dataframe(periods=periods, freq=f'{data_logging_interval}T')

    # Predict the future values for temp
    forecast_temp = model_temp.predict(future_temp)

    # Compute RMSE for temp
    predicted_temp = forecast_temp['yhat'].values[-len(test_temp):]
    rmse_temp = np.sqrt(mean_squared_error(test_temp['y'], predicted_temp))

    # Print RMSE value for temp
    print(f"RMSE for temp: {rmse_temp}")

    # Making predictions on the same data or new data
    X_predict = X_predict.dropna()  # Remove rows with NaN values in 'ds'
    predict_temp = model_temp.predict(X_predict.rename(columns={X_predict.columns[0]: 'ds'}))

    df_temp_pred = pd.DataFrame(data=predict_temp['yhat'].values, index=X_predict.index, columns=['temp_pred'])

    df = df_temp_pred

    # Print the head of the resulting DataFrame
    print(df.head())

    return df

# Sample data creation
data = {
    'ts': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'temp': np.random.randn(100),
    'new_point': np.random.randn(100)
}
df = pd.DataFrame(data)

# Introducing some NaNs to simulate missing data
df.loc[10:20, 'temp'] = np.nan

# Converting to the expected input format
df['his'] = df.apply(lambda x: pd.DataFrame([{'ts': x['ts'], 'temp': x['temp'], 'new_point': x['new_point']}]), axis=1)

# Define the values for length_of_missing_data and data_logging_interval
length_of_missing_data = 400  # example value
data_logging_interval = 1  # example value

# Call the forecasting function with these parameters
forecast_df = prophet_forecasting(pd.DataFrame([{'his': df}]), length_of_missing_data, data_logging_interval, dqStart=None)

# Check the results
print(forecast_df.head())


C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


KeyError: "None of ['ts'] are in the columns"